In [73]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPooling2D,concatenate,AveragePooling2D,Dropout,Dense,Softmax
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import tensorflow_datasets as tfds
import cv2
import numpy as np

In [83]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    'data/train',
    seed=123,
    batch_size=32,
    image_size=(224,224))


test_ds = tf.keras.utils.image_dataset_from_directory(
    'data/test/',
    seed=123,
    image_size=(224,224))


val_ds = tf.keras.utils.image_dataset_from_directory(
    'data/valid/',
    seed=123,
    image_size=(224,224))

Found 2339 files belonging to 10 classes.
Found 50 files belonging to 10 classes.
Found 50 files belonging to 10 classes.


In [99]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [84]:
train_ds.class_names

['AFRICAN LEOPARD',
 'CARACAL',
 'CHEETAH',
 'CLOUDED LEOPARD',
 'JAGUAR',
 'LIONS',
 'OCELOT',
 'PUMA',
 'SNOW LEOPARD',
 'TIGER']

In [100]:
class Inception(keras.layers.Layer):
    def __init__(self,filter_1,filter_reduce_3x,filter_3x,filter_reduce_5x,filter_5x,pool_proj):
        super(Inception,self).__init__()

        self.block1 = Conv2D(filter_1,kernel_size=(1,1),padding='same',activation='relu')

        self.block2_1 = Conv2D(filter_reduce_3x,kernel_size=(1,1),padding='same',activation='relu')
        self.block2_2 = Conv2D(filter_3x,kernel_size=(3,3),padding='same',activation='relu')

        self.block3_1 = Conv2D(filter_reduce_5x,kernel_size=(1,1),padding='same',activation='relu')
        self.block3_2 = Conv2D(filter_5x,kernel_size=(5,5),padding='same',activation='relu')

        self.block4_1 = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same')
        self.block4_2 = Conv2D(pool_proj,kernel_size=(1,1),padding='same',activation='relu')

    def call(self,x):
        layer_1 = self.block1(x)

        layer_2 = self.block2_1(x)
        layer_2 = self.block2_2(layer_2)

        layer_3 = self.block3_1(x)
        layer_3 = self.block3_2(layer_3)

        layer_4 = self.block4_1(x)
        layer_4 = self.block4_2(layer_4)

        return concatenate([layer_1,layer_2,layer_3,layer_4],axis=-1)
    
class GoogleNet(keras.Model):
    def __init__(self):
        super(GoogleNet,self).__init__()

        self.pool = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')

        #block 1
        self.conv1 = Conv2D(64,kernel_size=(7,7),strides=(2,2),padding='same',activation='relu')
        
        #block 2
        self.conv2 = Conv2D(64,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')

        self.conv3 = Conv2D(192,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')

        #block 3 (inception 3)
        self.inception_3a = Inception(64,96,128,16,32,32)
        self.inception_3b = Inception(128,128,192,32,96,64)

        #block 4 (inception 4)
        self.inception_4a = Inception(192,96,208,16,48,64)
        self.inception_4b = Inception(160,112,224,24,64,64)
        self.inception_4c = Inception(128,128,256,24,64,64)
        self.inception_4d = Inception(112,144,288,32,64,64)
        self.inception_4e = Inception(256,160,320,32,128,128)

        #block 5 (inception 5)
        self.inception_5a = Inception(256,160,320,32,128,128)
        self.inception_5b = Inception(384,192,384,48,128,128)

        #block 6
        self.avg_pool = AveragePooling2D(pool_size=(7,7),strides=(1,1),padding='valid')
        self.dropout = Dropout(0.4)
        self.fc = Dense(10,activation='softmax')


    def call(self,x):

        x = self.conv1(x) #
        x = self.pool(x)

        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool(x)

        x = self.inception_3a(x)
        x = self.inception_3b(x)
        x = self.pool(x)

        x = self.inception_4a(x)
        x = self.inception_4b(x)
        x = self.inception_4c(x)
        x = self.inception_4d(x)
        x = self.inception_4e(x)
        x = self.pool(x)

        x = self.inception_5a(x)
        x = self.inception_5b(x)
        
        x = self.avg_pool(x)
        x = self.dropout(x)
        x = self.fc(x)

        return x

In [101]:
model = GoogleNet()

In [102]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [103]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [104]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [105]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [106]:
EPOCHS = 5
with tf.device('/device:GPU:0'):
    for epoch in range(EPOCHS):
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        for images, labels in train_ds:
            train_step(images, labels)
        for test_images, test_labels in test_ds:
            test_step(test_images, test_labels)
        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}')       

c:\Users\justf\miniconda3\envs\gpu\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 1, Loss: 2.3056039810180664, Accuracy: 9.210684776306152, Test Loss: 2.3029415607452393, Test Accuracy: 11.869436264038086
Epoch 2, Loss: 2.3033952713012695, Accuracy: 9.57986068725586, Test Loss: 2.302717924118042, Test Accuracy: 9.792284965515137
Epoch 3, Loss: 2.3030688762664795, Accuracy: 9.559797286987305, Test Loss: 2.3015007972717285, Test Accuracy: 8.75370979309082
Epoch 4, Loss: 2.302891731262207, Accuracy: 10.061395645141602, Test Loss: 2.3030121326446533, Test Accuracy: 9.792284965515137
Epoch 5, Loss: 2.3029286861419678, Accuracy: 9.539732933044434, Test Loss: 2.304098129272461, Test Accuracy: 10.830860137939453


In [110]:
model.summary()

Model: "google_net_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 max_pooling2d_90 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 conv2d_513 (Conv2D)         multiple                  9472      
                                                                 
 conv2d_514 (Conv2D)         multiple                  4160      
                                                                 
 conv2d_515 (Conv2D)         multiple                  110784    
                                                                 
 inception_81 (Inception)    multiple                  163696    
                                                                 
 inception_82 (Inception)    multiple                  388736    
                                                      

In [1]:
a=116+120+120+120+120+8+132+102+78+90
b=30+30+30+30+30+30+24+18+18
print(a/b)

4.191666666666666


In [ ]:
5 - 4.19 
4.5 - x